In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t
spark = SparkSession.builder.appName("Spark Streming").getOrCreate()
spark

In [19]:
df = spark.read.csv("Transection.csv",header = True, inferSchema=True)
# df.show()

In [20]:
df.tail(4)

[Row(step=743, type='TRANSFER', amount=6311409.28, nameOrig='C1529008245', oldbalanceOrg=6311409.28, newbalanceOrig=0.0, nameDest='C1881841831', oldbalanceDest=0.0, newbalanceDest=0.0, isFraud=1, isFlaggedFraud=0),
 Row(step=743, type='CASH_OUT', amount=6311409.28, nameOrig='C1162922333', oldbalanceOrg=6311409.28, newbalanceOrig=0.0, nameDest='C1365125890', oldbalanceDest=68488.84, newbalanceDest=6379898.11, isFraud=1, isFlaggedFraud=0),
 Row(step=743, type='TRANSFER', amount=850002.52, nameOrig='C1685995037', oldbalanceOrg=850002.52, newbalanceOrig=0.0, nameDest='C2080388513', oldbalanceDest=0.0, newbalanceDest=0.0, isFraud=1, isFlaggedFraud=0),
 Row(step=743, type='CASH_OUT', amount=850002.52, nameOrig='C1280323807', oldbalanceOrg=850002.52, newbalanceOrig=0.0, nameDest='C873221189', oldbalanceDest=6510099.11, newbalanceDest=7360101.63, isFraud=1, isFlaggedFraud=0)]

In [21]:
df.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [22]:
df = df.drop('isFraud','isFlaggedFraud')

In [23]:
df.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|
|   1| PAYMENT|  7817.71|  C90045638|      53860.0|      46042.29| M573487274|           0.0|           0.0|
|   1| PAYMENT|  71

#### Step maps a unit of time in the real world. In this case 1 step in 1 hour of time. so we can assume for the this example that we have another job that runs every hour and gets all the transections o in that time frame

In [25]:
df.groupBy('step').count().show(3)

+----+-----+
|step|count|
+----+-----+
|  12|36153|
|   1| 2708|
|  13|37515|
+----+-----+
only showing top 3 rows



In [26]:
steps = df.select('step').distinct().collect()

In [10]:
# from pyspark.sql import SparkSession

# # Assuming you have already created a SparkSession named 'spark'
# # spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# for step in steps[:]:
#     _df = df.filter(f"step = {step[0]}")  # Correcting the where clause

#     # Using repartition(1) to save the dataframe to a single CSV file
#     _df.repartition(1).write.mode("append").option("header", "true").csv(r"Data/Data")


In [35]:
!cd Data/Data && ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [37]:
part = spark.read.csv(
    # "Data/Data/part-00000-00391bf8-1870-4460-830f-3a9834b9ec22-c000.csv"
    "Data/Data/part-00000-0248b078-5b42-4741-a34d-41cbb7eca5e1-c000.csv"
    
                      ,header = True, inferSchema=True)
part.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
| 144| PAYMENT| 12248.73|C1175735940|      11670.0|           0.0|M1932164650|           0.0|           0.0|
| 144|CASH_OUT|140995.09|C1534849700|          0.0|           0.0| C960557720|    3141468.33|    3282463.42|
| 144|TRANSFER|612743.87| C160577338|    612743.87|           0.0| C162068560|           0.0|           0.0|
| 144|CASH_OUT|612743.87|C1830172898|    612743.87|           0.0|C1813501651|     311807.53|     924551.41|
| 144| PAYMENT|  4875.51|C1299042372|      48221.0|      43345.49|M1225254303|           0.0|           0.0|
| 144| PAYMENT|  5876.43|C1370763362|      30110.0|      24233.57| M221889438|           0.0|           0.0|
| 144|CASH_OUT| 333

In [38]:
part.groupBy('step').count().show(3)

+----+-----+
|step|count|
+----+-----+
| 144| 3523|
+----+-----+



### creating the streaming versoin of input

In [42]:
dataSchema = part.schema

In [44]:
dataSchema

StructType([StructField('step', IntegerType(), True), StructField('type', StringType(), True), StructField('amount', DoubleType(), True), StructField('nameOrig', StringType(), True), StructField('oldbalanceOrg', DoubleType(), True), StructField('newbalanceOrig', DoubleType(), True), StructField('nameDest', StringType(), True), StructField('oldbalanceDest', DoubleType(), True), StructField('newbalanceDest', DoubleType(), True)])

In [45]:
streaming = (
    spark.readStream.schema(dataSchema).option("maxFilesPerTrigger",1).csv("Data/Data")
)

In [48]:
dest_count = streaming.groupBy("nameDest").count().orderBy(f.desc("count"))